In [33]:
import requests
import json
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

sido_code ={
	"서울":1,
	"인천":2,
	"대전":3,
	"대구":4,
	"광주":5,
	"부산":6,
	"울산":7,
	"세종시":8,
	"경기":31,
	"강원":32,
	"충북":33,
	"충남":34,
	"경북":35,
	"경남":36,
	"전북":37,
	"전남":38,
	"제주":39}

In [1]:
sido_code ={
	"서울":1,
	"인천":2,
	"대전":3,
	"대구":4,
	"광주":5,
	"부산":6,
	"울산":7,
	"세종시":8,
	"경기":31,
	"강원":32,
	"충북":33,
	"충남":34,
	"경북":35,
	"경남":36,
	"전북":37,
	"전남":38,
	"제주":39
    }

In [2]:
sido_code = {v:k for k,v in sido_code.items()}

In [3]:
sido_code

{1: '서울',
 2: '인천',
 3: '대전',
 4: '대구',
 5: '광주',
 6: '부산',
 7: '울산',
 8: '세종시',
 31: '경기',
 32: '강원',
 33: '충북',
 34: '충남',
 35: '경북',
 36: '경남',
 37: '전북',
 38: '전남',
 39: '제주'}

In [4]:
sido = list(range(1,9))+ list(range(31,40))
sido

[1, 2, 3, 4, 5, 6, 7, 8, 31, 32, 33, 34, 35, 36, 37, 38, 39]

지역 검색
https://korean.visitkorea.or.kr/main/area.do

지역 코드별
https://korean.visitkorea.or.kr/main/cr_main.do?type=place&areacode=37


https://korean.visitkorea.or.kr/api/v1/curation/list?offset=5&page=1&device=PC&type=MAIN_A&regionCode=37&sequence=0&order=POPULAR&latitude=0.0&longitude=0.0&_=1700026774369


In [6]:
import pandas as pd
import numpy as np

In [66]:

result = pd.DataFrame()

for code in tqdm(sido[:1]):
    
    offset = 1
    url = f'https://korean.visitkorea.or.kr/api/v1/curation/list?offset={offset}&page=1&device=PC&type=MAIN_A&regionCode={code}&sequence=0&order=POPULAR&latitude=0.0&longitude=0.0&_=1700026774369'
    res = requests.get(url)
    group = json.loads(res.text)

    offset = group['data']['total']

    print('now_working :', sido_code[code], offset)
    url = f'https://korean.visitkorea.or.kr/api/v1/curation/list?offset={offset}&page=1&device=PC&type=MAIN_A&regionCode={code}&sequence=0&order=POPULAR&latitude=0.0&longitude=0.0&_=1700026774369'
    res = requests.get(url)
    group = json.loads(res.text)

    bag = []
    print('total_rows_number :', len(result))

    for data in group['data']['items']:

        try:
            city = sido_code[code]
            title = data['content']['title']
            overView = data['content']['databaseMaster']['overView']
            likeCount = data['content']['likeCount']
            shareCount = data['content']['shareCount']
            commentCount = data['content']['commentCount']
            cotId = data['content']['databaseMaster']['cotId']
            addr1 = data['content']['databaseMaster']['addr1']
            mapx = data['content']['databaseMaster']['mapx']
            mapy = data['content']['databaseMaster']['mapy']


            url2 = 'https://korean.visitkorea.or.kr/call'
            data = {
                    'cmd': 'TOUR_CONTENT_BODY_DETAIL',
                    'cotId': cotId,
                    'locationx': 0.0,
                    'locationy': 0.0,
                    'stampId': '1589345b-b030-11ea-b8bd-020027310001'   
            }
            res = requests.post(url2,data=data)
            group = json.loads(res.text)

            infoCenter = group['body']['detail']['infoCenter']
            parking = group['body']['detail']['parking']
            useTime = group['body']['detail']['useTime']
            tel = group['body']['detail']['tel']
            tagName = group['body']['detail']['tagName']
            tagName = ','.join([i for i in tagName.split('|') if not str.isdigit(i[0])])
            overView = re.sub(r'<.*?>|&lt;|&gt;|\n', '', overView)
            useTime = re.sub(r'<.*?>|&lt;|&gt;|\n', '', useTime)

            append_dict ={
                'city' : city,
                'cityCode': code,
                'title': title,
                'overView': overView,
                'likeCount': likeCount,
                'shareCount': shareCount,
                'commentCount': commentCount,
                # 'cotId': cotId,
                'addr': addr1,
                'infoCenter': infoCenter,
                'parking': parking,
                'useTime': useTime,
                'tel': tel,
                'tagName': tagName
                        }
            
            if group['body']['subArticle']:
                for i in group['body']['subArticle']:
                    append_list = []
                    content = i['contentBody']
                    content = re.sub(r'<.*?>|&lt;|&gt;|\n', '', content)
                    text = i['displayTitle'] + ':' + content
                    append_list.append( text )
            append_dict['detail'] = ','.join(append_list)
            append_dict['lat'] = mapx
            append_dict['lon'] = mapy

            bag.append(append_dict)
        except Exception as e:
            print(e)
            continue

    result = pd.concat([result, pd.DataFrame(bag)], ignore_index=True )



    

  0%|          | 0/1 [00:00<?, ?it/s]

now_working : 서울 160
total_rows_number : 0


100%|██████████| 1/1 [00:16<00:00, 16.12s/it]


In [67]:
result


,city,cityCode,title,overView,likeCount,shareCount,commentCount,addr,infoCenter,parking,useTime,tel,tagName,detail,lat,lon
0,서울,1,서울숲,2005년 6월 18일 ‘서울숲’ 개원 이전까지는 임금의 사냥터였고 1908년 설치...,94,49,162,서울특별시 성동구 뚝섬로 273,02-460-2905,주차가능,"* 생태숲(사슴우리, 보행가교)- 05:30~21:30 (생태보전구역)* 곤충식물원...",033-738-3588,"가족여행,걷기길,관광지,데이트코스,반려동물동반여행지,생태관광지,서울숲,수도권,수도권...","동반 시 필요사항:입마개 착용,목줄 착용",127.0417770333,37.5430553337
1,서울,1,길상사(서울),"서울 성북동 중턱에 위치한 길상사는 예전 대원각 자리에 세워진 사찰로, 대원각 주인...",13,9,8,서울특별시 성북구 선잠로5길 68,02-3672-5945,"있음(단, 주차장이 혼잡하니 되도록이면 대중교통 이용)",,033-738-3588,"관광지,길상사(서울),불교,불교문화,사찰,사찰여행,산사,수도권,역사,연중무휴,자연속...",한국어 안내서비스:가능,126.9946241836,37.5990951960
2,서울,1,낙산공원,낙산(125m)은 서울의 내사산의 하나로 좌청룡에 해당하는 산이다. 지형이 낙타의 ...,24,23,9,서울특별시 종로구 낙산길 41,02-743-7985~6,있음,낙산공원 상시개방낙산전시관 09:00~17:00,033-738-3588,"가족여행,관광지,나들이,낙산공원,데이트코스,드라이브여행,서울야경,성곽걷기,수도권,아...","이용가능시설:[기반시설] 관리도로 2,920m, 산책로 1,010m, 광장5개소[교...",127.0065125148,37.5805725621
3,서울,1,경복궁,경복궁은 1395년 태조 이성계에 의해서 새로운 조선왕조의 법궁으로 지어졌다. 경복...,84,94,66,서울특별시 종로구 사직로 161,경복궁 관리소 02-3700-3900,가능,하절기 09:00~18:30동절기 09:00~17:00※ 자세한 사항은 홈페이지 참조,033-738-3588,"BTS,bts여행,bts투어,가족여행,경복궁,고궁,관광지,교과서속여행,당일치기여행,...",외국어 안내서비스:※ 시작하는 곳 : 경복궁 안내실 앞(흥례문 안쪽)※ 소요시간 :...,126.9769930325,37.5788222356
4,서울,1,경동시장,6.25전쟁 이후 서울 사람들의 생활이 회복되기 시작하면서 경기도 북부와 강원도 등...,13,4,184,서울특별시 동대문구 고산자로36길 3,02-967-8721,"주차 가능(유료)<br>30분 1,000원 (30분 경과 후 15분당 500원)",점포별 상이,033-738-3588,"가볼래터,가족여행,나들이,나홀로여행,데이트코스,먹거리,서울경동시장,쇼핑,수도권,시장...",외국어 안내서비스:※ 시작하는 곳 : 경복궁 안내실 앞(흥례문 안쪽)※ 소요시간 :...,127.0391568047,37.5790899880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,서울,1,마포아트센터,"2008년 개관한 마포아트센터는 홍대, 신촌, 서강대 등 대학가에 둘러싸인 새로운 ...",0,0,0,서울특별시 마포구 대흥로20길 28,02-3274-8500,주차 가능,공연에 따라 다름,033-738-3588,"가족체험여행,마포아트센터,문화시설,상설공연,실내여행지,아이와함께,연인과함께,예술,친...","사업내용:공연장, 스포츠센터, 아카데미 운영 및 지역연계 사업",126.9454967222,37.5498944782
156,서울,1,명보아트홀,"서울의 중심, 을지로3가에 위치한 명보아트홀은 전문공연장으로 이루어진 복합문화공간이...",1,0,0,서울특별시 중구 마른내로 47,02-2274-2121,,공연에 따라 다름,033-738-3588,"도심여행,명보아트홀,문화시설,복합문화공간,상설공연,예술","대관안내:* 대관 대상 : 뮤지컬, 오페라, 연극, 콘서트, 무용 등 공연을 목적으...",126.9931808474,37.5646797260
157,서울,1,우리금융아트홀,올림픽공원 우리금융아트홀(올림픽 역도경기장)은 국민체육진흥공단이 순수문화예술 공연발...,0,0,1,서울특별시 송파구 올림픽로 424,02-2180-3564~6,주차 가능,공연에 따라 다름,033-738-3588,"데이트코스,문화공간,문화시설,문화행사,복합문화공간,사계절,상설공연,수도권,아이와함께...","대관안내:* 대관 대상 : 뮤지컬, 오페라, 연극, 콘서트, 무용 등 공연을 목적으...",127.1243145673,37.5218287295
158,서울,1,KT&G 상상마당(홍대),"* 복합문화공간 상상마당 설립 취지문화예술인과 대중을 위한 매개체로, 신진 예술가들...",0,1,1,서울특별시 마포구 어울마당로 65,02-330-6227,,시설별로 다름,033-738-3588,"KT&G상상마당,나홀로여행,도심여행,문화시설,문화예술,복합문화공간,연인과함께,열린문...","대관안내:아카데미, 라이브홀, 시네마 대관 가능",126.9210433658,37.5509355861
